# Utilizing Azure Open AI + MS SQL with Langchain for Larger Databases

When working with databases that have numerous tables and rows, inputting the entire schema into a Language Model (LLM) prompt can become cumbersome and impractical. To address this issue, the Langchain library offers a solution designed to retrieve the schema information only for the tables relevant to the user's query. This enables efficient interaction with the database using Azure Open AI and MS SQL.

## How Langchain Works

1. **User Input Analysis**: Langchain begins by analyzing the user's input along with the database table names provided.

2. **Schema Extraction**: After understanding the user query, Langchain identifies the specific tables that are relevant to the query. It then extracts the schema information for these relevant tables.

3. **Context Generation**: Langchain constructs a context that includes the extracted schema information. This context is then combined with a system message, which instructs the Language Model (LLM) to convert natural language into SQL.

4. **SQL Query Translation**: With the schema information and the system message, a standard LLM (GPT4) is now equipped to translate the user's natural language query into SQL code.

5. **Execution on the Server**: The generated SQL code is executed on the server-side, allowing for efficient and accurate database interactions.

6. **Result Retrieval and Display**: Finally, the results of the SQL query are retrieved from the database server and presented to the user.

## Benefits of Using Langchain

- **Efficiency**: Langchain streamlines the process of working with large databases by focusing on relevant schema information, saving time and resources.

- **Clarity**: Users can communicate with the database using natural language queries, enhancing the accessibility of database interactions.

- **Accuracy**: By providing context and relevant schema data, Langchain helps ensure that the SQL queries generated by the LLM are precise and meaningful.

This integration of Azure Open AI, MS SQL, and Langchain offers a powerful solution for effectively handling larger databases while maintaining a user-friendly and efficient experience.


## Imports and Server Setup

In [1]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain.chains import create_sql_query_chain
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import AzureChatOpenAI #Use Azure OpenAI for secure chat
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.agent_types import AgentType
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings #Use Azure OpenAI for secure embeddings
from langchain_community.agent_toolkits.sql.base import create_sql_agent

import os
import openai
import pyodbc
import re
import ast
import ast
import re
import pandas as pd
import nest_asyncio

nest_asyncio.apply()

from sqlalchemy import create_engine
from operator import itemgetter
from decimal import Decimal
from tabulate import tabulate
import numpy as np


In [2]:

# OpenAI configuration
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = "2023-12-01-preview"


os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""


# Database connection parameters
server = ''
database = ''
username = ''
password = ''
driver = 'ODBC+Driver+18+for+SQL+Server'
driver2 = '{ODBC Driver 18 for SQL Server}' ## change according to your driver

# sqlalchemy connection needed for retrieving relevant tables 
db_uri = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'
engine = create_engine(db_uri)

db = SQLDatabase(engine)

In [3]:
# Initialize the large language model
llm = AzureChatOpenAI(
    openai_api_version="2023-12-01-preview",
    azure_deployment="mdxgpt4",
    temperature= 0,
    openai_api_type= AgentType.OPENAI_FUNCTIONS
    
)


# Extracting the relevant tables from the database

Leverage GPT to retrieve the most relevant tables from a database based on a given query and context. This approach is especially beneficial in environments with numerous tables, as it streamlines the process of interacting with large databases. Instead of passing the entire database schema—which could exceed the input token limit of the model—this method focuses on first identifying and retrieving the tables most pertinent to the query. By doing so, it minimizes the amount of schema information required as input, making the process more efficient and manageable. This targeted retrieval ensures that only the necessary data is processed, leading to quicker and more accurate responses from GPT

In [4]:
# Define a Pydantic model for tables
class Table(BaseModel):
    name: str 

# Get usable table names from your database
table_names = "\n".join(db.get_usable_table_names())

# System message for extraction chain
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
The tables are:

{(table_names)}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

# Create an extraction chain for table names
table_chain = create_extraction_chain_pydantic(Table, llm, system_message=system)


In [5]:
def get_table_names(table_list) -> str:
    names = []
    for item in table_list:
        names.append(item.name)
    return names


In [6]:
table_chain = table_chain| get_table_names

In [7]:
table_chain = {"input": itemgetter("question")} | table_chain

In [8]:

schema_query = ''' SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE 
                FROM INFORMATION_SCHEMA.COLUMNS
                ORDER BY TABLE_NAME, ORDINAL_POSITION; '''



def get_table_schema(table_list):
    schema = []
    for item in table_list:
        schema.append(db.get_table_schema(item))
    return schema

# Vector Databse for Hight Cardinality Columns


A vector database for high cardinality columns can be helpful during the querying process by comparing top matching words and potentially replacing any misspellings. Let's break down how it works:

1. Vector Database: A vector database is a data structure that stores vectors, which are mathematical representations of data points. In this case, the vectors represent words or phrases in the columns with high cardinality, such as proper nouns, keywords, or text columns.

2. High Cardinality Columns: High cardinality columns refer to columns that have a large number of distinct values. For example, a column containing names of people or products may have high cardinality because there can be thousands or even millions of unique names.

3. Top Matching Words: When querying the vector database, you can compare the input query with the vectors stored in the database. By using techniques like cosine similarity or nearest neighbor search, you can identify the top matching words or phrases that are most similar to the query. This allows you to find relevant matches even if there are slight variations or misspellings in the input.

4. Replacing Misspellings: One of the advantages of using a vector database is its ability to handle misspellings. By comparing the query with the vectors in the database, you can identify potential misspellings and suggest alternative words or phrases that are similar to the query. This can help improve the accuracy of the search results and provide better user experience.

Overall, a vector database for high cardinality columns provides a powerful tool for efficient querying and handling misspellings. It leverages vector representations and similarity measures to find top matching words and suggest replacements for misspelled queries, improving the accuracy and effectiveness of the search process.

In [10]:

noun_extraction_query = """
SELECT TABLE_NAME, COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE DATA_TYPE IN ('nvarchar', 'varchar')
AND TABLE_NAME <> 'database_firewall_rules'
"""
# Use db.run instead of cursor.execute
result = ast.literal_eval(db.run(noun_extraction_query))


# Store unique values from each column in this list
proper_nouns = []


if result:  # Check if the result list is not empty
    for row in result:
        table_name = row[0]
        column_name = row[1]

        # Query to get all values from the column
        query = f"SELECT {column_name} FROM {table_name}"

        # Use db.run instead of cursor.execute
        result = ast.literal_eval(db.run(query))

        # Check if the majority of the values are numerical
        numerical_values = [value for value in result if re.match(r'^\d+$', str(value[0]))]
        if len(numerical_values) > len(result) / 2:
            continue  # Skip this column if the majority of the values are numerical
        print(column_name)
        # Query to get all unique values from the column
        query = f"SELECT DISTINCT {column_name} FROM {table_name}"

        # Use db.run instead of cursor.execute
        result = ast.literal_eval(db.run(query))

        # Add the unique values to the list
        proper_nouns.extend([value[0] for value in result])

Company
Customer
Customer_Regeion
Sales_Territory
SalesRep_Name_1
PO
PG
Part_Number
RM_FG
Part_Description
Full_PG_2
PG_2
Ship_to_Name
Ship_To_State
Ship_To_Country
Ship_to_Sales_Rep
Open_Order
Open_Rel
TYPE
Company
Customer
Customer_Region
Sales_Territory
SalesRep_Name_1
Credit_Memo
PG
Part_Number
RM_FG
Line_Description
Full_PG_2
PG_2
GL_Account
PO
Ship_To_Name
Ship_To_State
Ship_To_Country
Ship_To_Sales_Rep
TYPE
Consolidated_Customer_Name
Ship_To_Name
Part_Number
Consolidated_Customer_Name


In [14]:
proper_nouns = list(set(proper_nouns))  # Remove duplicates
pd.DataFrame(proper_nouns).to_csv('Proper_Nouns.csv', index=False, header=False, encoding='utf-8')

In [ ]:
proper_nouns = pd.read_csv('Proper_Nouns.csv', encoding='utf-8', header=0)
len(proper_nouns)

In [6]:
#Set up openai embeddings

embedder = AzureOpenAIEmbeddings(
    openai_api_version="2023-12-01-preview",
    azure_deployment="text_model",
    openai_api_type= AgentType.OPENAI_FUNCTIONS
)


In [36]:
# Define batch size
batch_size = 500  # Adjust the batch size based on your needs and system capabilities

# Split the proper_nouns list into batches
batches = [proper_nouns[i:i + batch_size] for i in range(0, len(proper_nouns), batch_size)]

# Initialize an empty list to hold all embeddings
all_embeddings = []


batch_number = 0
#create vector from first row, save then add the rest of the rows
vector_db = FAISS.vector_store(batches[0], embedder)
vector_db.save_local("faiss_index")

for batch in batches[1:]:
    await vector_db.aadd_texts(batch)
    vector_db.save_local(f"faiss_index")
    print(f'Processed batch {batch_number}')
    batch_number += 1

Processed batch 0
Processed batch 1
Processed batch 2
Processed batch 3
Processed batch 4
Processed batch 5
Processed batch 6
Processed batch 7
Processed batch 8
Processed batch 9
Processed batch 10
Processed batch 11
Processed batch 12
Processed batch 13
Processed batch 14
Processed batch 15
Processed batch 16
Processed batch 17
Processed batch 18


In [37]:
vector_db = FAISS.load_local('faiss_index', embedder, allow_dangerous_deserialization=True)
retriever = vector_db.as_retriever(search_kwargs={"k": 15})

# System Prompt

In [38]:
sys_prompt = '''
        As an MS SQL expert, please create a syntactically correct MS SQL query in response to the given input question. Adhere to the following guidelines:

        - List specific column names instead of using "SELECT *", even when all columns are needed.
        - Use only read-only operations (e.g., SELECT) and avoid commands that modify the database (like UPDATE, CREATE TABLE). (most important!)
        - For the current date, use CAST(GETDATE() as date).
        - Focus on the provided table and column information. Use columns that exist in the specified tables.
        - Replace 'LIMIT' with the 'TOP' clause
        - It is crucial to format numerical results with two decimal places and a thousands separator. Please ensure to use the 'FORMAT' function in SQL for formatting currency and numerical outputs, especially in computations like summing or subtracting prices and costs. (important!)
        - If result is expected to be currency such as profit, or sales price, please format the result as currency with a dollar sign, thousands separator, and two decimal places.
        - Exclude extraneous explanations; only return the query in MS SQL syntax. Your must be a runnable sql query!
        - Use SUM insteead of SELECT COUNT(*) for numerical aggregations.
        - If you need to select rows use the headers provided in the schema_info instead of using SELECT *.
        - Format quantity values with a thousands separator.
        - When translating a natural language query into SQL, particularly when the request involves calculating rankings or comparisons for each year or month, you should utilize the PARTITION BY clause within window functions instead of solely using GROUP BY. 
          The PARTITION BY clause is essential for dividing the dataset into distinct partitions (like per year or per month) to perform calculations independently within each partition. This approach is crucial for scenarios where you need to identify top performers, trends, or comparisons within each time segment.
          For aggregate calculations that summarize data across the whole dataset, continue to use GROUP BY"
        - Limit the number of rows returned by the query to {top_k} unless instructed otherwise.
                --------------------------------------------------------------------------------

    Tables and Columns to be considered:
        {schema_info}



        (Note: Do not include 'database_firewall_rules', 'sysdiagrams', or any tables with 'aspnet' in their names.)
        -------------------------------------------------------------------------------------------------
        Cross-verify the spelling of nouns against this provided, though non-exhaustive, list: {proper_nouns}.
        Correct the spelling of company names, product names/ parts, or other porper nouns when constructing the query by making use of the provided list of keywords.

        Your response should exclusively contain the MSSQL query in the correct syntax, utilizing 'TOP' instead of 'LIMIT', without any extra explanations or text.

        '''

In [61]:

prompt = ChatPromptTemplate.from_messages([("system", sys_prompt), ("human", "{input}")])

In [65]:
sql_agent = create_sql_agent(llm=llm, db=db, agent_type="openai-tools", verbose=True)
query_chain = create_sql_query_chain(llm, db,prompt=prompt)
RunnablePassthrough.assign(table_names_to_use=table_chain)|query_chain

In [66]:
retriever_chain = (
    itemgetter("question")
    | retriever
    | (lambda docs: "\n".join(doc.page_content for doc in docs))
)
chain = RunnablePassthrough.assign(proper_nouns=retriever_chain) | query_chain

In [67]:
chain.invoke({"question": "What is the total sales for each customer?"})

'```sql\nSELECT TOP 5 CAST(Customer AS NVARCHAR(MAX)) AS Customer, SUM(CAST(USD_Total_Price AS MONEY)) AS TotalSales\nFROM [CURRENT_SALES]\nGROUP BY Customer\n```'

## Run Query 

In [146]:
def run_query(user_query):
    query = query_chain.invoke({"question": f'{user_query}',"proper_nouns": ""})
    query =  query.replace("```sql", "").replace("```", "").strip()
    result = db.run(query)
    result = re.sub(r"Decimal\('([^']*)'\)", r"\1", result)
    result = ast.literal_eval(result)
    result = tabulate(result, headers='keys', tablefmt='pretty_grid')
    return "SQL Query: \n" + query + "\n\n" + "Result: \n" + result

In [148]:
print(run_query("What are the total sales for each company for each year?"))

SQL Query: 
SELECT TOP 5 Company, Year, SUM(USD_Total_Price) AS Total_Sales
FROM CURRENT_SALES
GROUP BY Company, Year

Result: 
0        1            2
----  ----  -----------
10US  2017  4.66686e+07
20BV  2017  1.08592e+07
30CH  2017  2.66865e+06
10US  2018  4.21011e+07
20BV  2018  1.00496e+07



# Conclusion

In this notebook, we have demonstrated how to use the Langchain library in conjunction with Azure OpenAI and MS SQL to efficiently interact with large databases. The Langchain library provides a streamlined approach to extract schema information only for the tables relevant to the user's query, making it practical to work with databases that have numerous tables and rows.

The notebook uses several Python libraries and frameworks, including Pydantic for data validation and settings management, SQLAlchemy for SQL toolkit and Object-Relational Mapping, and OpenAI for AI models. It also uses the AzureChatOpenAI class from the Langchain library to initialize the large language model.

The Langchain library, combined with Azure OpenAI and MS SQL, offers a powerful solution for effectively handling larger databases. It allows users to communicate with the database using natural language queries, enhancing the accessibility of database interactions. By providing context and relevant schema data, Langchain helps ensure that the SQL queries generated by the Language Model are precise and meaningful.

In real world applications, this could be used to allow no technical employees or clients query large sql databases effortlessly, empowering them to make more data driven decisions.

Overall, the integration of Azure OpenAI, MS SQL, and Langchain offers a user-friendly and efficient experience for effectively handling larger databases.